In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from os.path import join

import tensorflow as tf 
from tensorflow.keras import layers, models, optimizers, Input, Model
from tensorflow.keras.utils import to_categorical

In [16]:
datax = np.load(join('data', 'MNIST.npy'))
datay = np.load(join('data', 'Label.npy'))

In [17]:
datax_norm = datax/255
print('최대 : {}, 최소 : {}'.format(np.max(datax_norm), np.min(datax_norm)))

최대 : 1.0, 최소 : 0.0


In [18]:
datay_onehot = to_categorical(datay)
print(datay[0:10])
print(datay_onehot[0:10,:])

[5 0 4 1 9 2 1 3 1 4]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [19]:
from sklearn.model_selection import train_test_split
trnx, tstx, trny, tsty = train_test_split(datax_norm, datay_onehot, test_size = 0.3, random_state = 111)

In [20]:
print(trnx.shape)
print(trny.shape)

(42000, 28, 28)
(42000, 10)


In [21]:
trnx_reshape = trnx.reshape(-1,28,28,1)
tstx_reshape = tstx.reshape(-1,28,28,1)
trnx_reshape.shape

(42000, 28, 28, 1)

In [22]:
def res_identity(x, filters):
    x_skip = x
    f1, f2 = filters
    
    x = layers.Conv2D(f1, kernel_size = (1, 1), strides=(1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2D(f1, kernel_size = (3, 3), strides=(1, 1), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2D(f2, kernel_size = (1, 1), strides=(1, 1), padding = 'valid')(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Activation("relu")(x)
    
    x = layers.Add()[x,x_skip]
    x = layers.Activation("relu")(x)
    
    return x

In [23]:
def res_conv(x, s, filters):
    x_skip = x
    f1, f2 = filters
    
    x = layers.Conv2D(f1, kernel_size=(1,1), strides=(s,s), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(f1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f2, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    
    x_skip = layers.Conv2D(f2, kernel_size=(1,1), strides=(s,s), padding='valid')(x_skip)
    x_skip = layers.BatchNormalization()(x_skip)
    
    x = layers.Add()[x, x_skip]
    x = layers.Activation("relu")(x)
    
    return x

In [12]:
def resnet50():
    input_im = input(shape=(trnx_reshape.shape[1], trnx_reshape.shape[2], trnx_reshape.shape[3]))
    x = layers.ZeroPadding2D(padding=(3, 3))(input_im)

    x = layers.Conv2D(64, kernel_size=(7,7), strides=(2,2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2,2))(x)
    
    x = res_conv(x, s=1, filters=(64,256))
    x = res_identity(x, filters=(64,256))
    x = res_identity(x, filters=(64,256))
    
    x = res_conv(x, s=2, filters=(128,512))
    x = res_identity(x, filters=(128,512))
    x = res_identity(x, filters=(128,512))
    x = res_identity(x, filters=(128,512))

    x = res_conv(x, s=2, filters=(256,1024))
    x = res_identity(x, filters=(256,1024))
    x = res_identity(x, filters=(256,1024))
    x = res_identity(x, filters=(256,1024))
    x = res_identity(x, filters=(256,1024))
    x = res_identity(x, filters=(256,1024))
    
    x = res_conv(x, s=2, filters=(512,2048))
    x = res_identity(x, filters=(512,2048))
    x = res_identity(x, filters=(512,2048))
    
    x = layers.AveragePooling2D((2,2), padding='same')(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(trny.shape[1], activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Mode(inputs=input_im, outputs=x, name='Resnet50')
    
    return model

In [ ]:
resnet_model = resnet50()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/questionnote/miniconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_9215/287772904.py", line 1, in <module>
    resnet_model = resnet50()
  File "/tmp/ipykernel_9215/2533550262.py", line 2, in resnet50
    input_im = input(shape=(trnx_reshape.shape[1], trnx_reshape.shape[2], trnx_reshape.shape[3]))
TypeError: raw_input() got an unexpected keyword argument 'shape'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/questionnote/miniconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2088, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most rece

In [9]:
input_shape = (28, 28, 1)

cnn_model = models.Sequential()

cnn_model.add(layers.Conv2D(16, (2,2), padding = 'same', input_shape = input_shape))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.MaxPooling2D((2,2)))

cnn_model.add(layers.Conv2D(32, (2,2), padding = 'same'))
cnn_model.add(layers.BatchNormalization())
cnn_model.add(layers.Activation("relu"))
cnn_model.add(layers.Dropout(0.2))
cnn_model.add(layers.MaxPooling2D((2,2)))

cnn_model.add(layers.Flatten())

cnn_model.add(layers.Dense(units = 128, activation = 'relu'))
cnn_model.add(layers.Dense(units = 10, activation = 'softmax'))

cnn_model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

2022-05-04 13:11:02.955850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-04 13:11:03.007311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-04 13:11:03.007516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 computeCapability: 8.6
coreClock: 1.845GHz coreCount: 20 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 208.65GiB/s
2022-05-04 13:11:03.007812: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-04 13:11:03.010067: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-04 13:11:03.012770: I tensorflow/stream_executor/pla

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        80        
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        2080      
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0

In [10]:
trnx_reshape = trnx.reshape(-1,28,28,1)
tstx_reshape = tstx.reshape(-1,28,28,1)
trnx_reshape.shape

(42000, 28, 28, 1)

In [1]:
history = cnn_model.fit(trnx_reshape, trny, validation_data = [tstx_reshape, tsty], batch_size = 5, epochs = 5)

NameError: name 'cnn_model' is not defined

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

print(history.history['val_accuracy'][-1])
print(np.max(history.history['val_accuracy']))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/questionnote/miniconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_7787/350508954.py", line 1, in <module>
    plt.plot(history.history['accuracy'])
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/questionnote/miniconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2088, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/questionnote/miniconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinner